In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

datdir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/cty_lst_/gpw4'
loadpath = f"{datdir}/downtown_sorted.txt"
df_loaded = pd.read_csv(loadpath, sep=',', header=None)
df_loaded.columns = df_loaded.iloc[0]
df_loaded = df_loaded[1:].reset_index(drop=True)
col_lst = ['wup', 'population', 'water availability', 'MPCY', 'potential']
for col in col_lst:
    df_loaded[col] = pd.to_numeric(df_loaded[col], errors='coerce')
df_loaded.head(10)

,city_num,city_name,scarcity,wup,population,water availability,MPCY,potential,region,country
0,1,Tokyo,●,37.0,32.0,14.000,450.0,-18.0,eastern asia,Japan
1,2,Delhi,●,22.0,22.0,4.400,200.0,-18.0,southern asia,India
2,4,Ciudad de México (Mexico City),●,20.0,20.0,0.300,15.0,-20.0,central america,Mexico
3,9,Al-Qahirah (Cairo),○,17.0,17.0,710.000,42000.0,690.0,northern africa,Egypt
4,11,Dhaka,●,15.0,15.0,0.096,6.4,-15.0,southern asia,Bangladesh
5,3,Shanghai,○,20.0,15.0,16.000,1100.0,1.9,eastern asia,China
6,13,Kolkata (Calcutta),○,14.0,14.0,32.000,2200.0,17.0,southern asia,India
7,8,Mumbai (Bombay),●,18.0,14.0,9.300,650.0,-5.0,southern asia,India
8,10,Beijing,●,16.0,14.0,2.400,170.0,-11.0,eastern asia,China
9,12,Buenos Aires,●,14.0,13.0,0.700,53.0,-13.0,south america,Argentina


In [16]:
region_counts = df_loaded['region'].value_counts()
wup_sum = df_loaded.groupby('region')['wup'].sum()
summary = pd.DataFrame({
    'Count': region_counts,
    'WUP Total Population': wup_sum
})
sorted_summary = summary.sort_values(by='WUP Total Population', ascending=False)
print(sorted_summary)
total_population_2010 =df_loaded['wup'].sum()
print(f"全都市の合計人口 (million person): {total_population_2010}")

                    Count  WUP Total Population
eastern asia          457                569.34
southern asia         197                276.10
northern america      161                204.79
south america         106                177.53
western asia           69                101.71
south-eastern asia     71                 97.94
eastern europe        101                 81.85
central america        57                 72.82
western africa         63                 59.61
eastern africa         47                 51.58
western europe         58                 51.39
southern europe        52                 51.36
northern africa        28                 46.05
northern europe        37                 35.02
middle africa          18                 28.05
oceania                15                 17.62
caribbean               9                 11.75
central asia            7                  6.47
全都市の合計人口 (million person): 1940.9799999999998


In [19]:
# グレースケールの色を設定
def get_gray_shade(value, min_value, max_value):
    gray_scale = 1 - (value - min_value) / (max_value - min_value)
    return (gray_scale, gray_scale, gray_scale)

# 最小値と最大値を取得
df_map = df_loaded.copy()
min_population = df_map['WUP Total Population'].min()
max_population = df_map['WUP Total Population'].max()

# 地域ごとのグレースケール色を設定
df_map['Color'] = df_map['WUP Total Population'].apply(get_gray_shade, args=(min_population, max_population))

# 地図の描画
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# 地図の基本要素を追加
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# 各地域を色分け
for _, row in df_map.iterrows():
    if row['Region'] == 'northern america':
        ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'subunits', '50m',
                        edgecolor='black', facecolor=row['Color'], zorder=0))
    # 他の地域の描画（例として書いていますが、各地域に対して同様の操作が必要です）

plt.title('World Population by Region (Grayscale)')
plt.show()


KeyError: 'WUP Total Population'